In [1]:
import pandas as pd
import requests as r
from bs4 import BeautifulSoup

In [2]:
nasa_url = "https://mars.nasa.gov/news/"
twitter_url = "https://twitter.com/marswxreport?lang=en"
facts_url = "http://space-facts.com/mars/"
astro_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"